In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
df=pd.read_csv('C:/Users/Dell/Desktop/Interview.csv')

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import feature_selection
from sklearn import tree
from sklearn import utils
from sklearn import naive_bayes
from sklearn import feature_selection
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import re


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 28 columns):
Date of Interview                                                                                     1233 non-null object
Client name                                                                                           1234 non-null object
Industry                                                                                              1233 non-null object
Location                                                                                              1233 non-null object
Position to be closed                                                                                 1233 non-null object
Nature of Skillset                                                                                    1233 non-null object
Interview Type                                                                                        1233 non-null object
Name(Cand ID)                   

#### Observations:
#### Looking into the data set , we can say that we have 1234 records with 28 columns/features
#### Firstly, we can clearly eleminate the last 5 columns , which have only null values
#### Secondly, we need to shorten and modify the column in order to make the data more interpretable
#### We need to clean the data before modelling

In [4]:
df.drop(["Unnamed: 23",'Unnamed: 24','Unnamed: 25','Unnamed: 26','Unnamed: 27'],axis=1,inplace=True)
df.columns=['date','client','industry','location','closedpos','skills','type','id','gender','currloc','jobloc','venue','natloc','permit','meeting','call','no.','resume','clear','letter','expat','obsat','marit']

# Date column


#### Converting the string format to date format 
#### Extracting year, month, day and day of week from the date column and appending the columns to the dataset

In [5]:
df.drop(df.index[df["date"].isnull()],axis=0,inplace=True)
df.date=df.date.str.replace(" ","")
df.date=df.date.str.replace(r'&\d[\d].\d\d\w\w','')
df.date=df.date.str.replace(r'&\d.\d\d\w\w','')
df.date=df.date.str.replace('/','.')
df.date=df.date.str.replace('-Apr-','.04.')
df.date=df.date.str.replace('–Apr-','.04.')
df.date=df.date.str.replace('Apr','.04.')
df.date=df.date.str.replace(r'\W16','.2016')
df.date=df.date.str.replace(r'\W15','.2015')
df.date=df.date.str.replace('-','.')
from datetime import datetime
df['date']=pd.to_datetime(df['date'],format='%d.%m.%Y')
df["date"]=df["date"].dt.date


In [ ]:
df["date"].unique()

In [ ]:
df2=pd.DataFrame({"year":[],"month":[],"day":[]})
df=pd.concat([df,df2],sort=False)
df2=pd.DataFrame({"dayofyear":[]})
df=pd.concat([df,df2],sort=False)
df2=pd.DataFrame({"dayofweek":[]})
df=pd.concat([df,df2],sort=False)
for i,s in df.iterrows():
    k=df["date"][i].timetuple()
    df["year"][i]=k.tm_year
    df["month"][i]=k.tm_mon
    df["day"][i]=k.tm_mday
    df["dayofweek"][i]=datetime.isoweekday(df["date"][i])
    df.dayofyear[i]=k.tm_yday

In [ ]:
df.info()

# Observed attendance column

In [ ]:
df.obsat.replace({'No':'N','Yes':'Y','yes':'Y','no':'N','yes ':'Y','No ':'N','NO':'N','no ':'N'},inplace=True)
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
df.loc[:,'obsat']=le.fit_transform(df.loc[:,'obsat'])

In [ ]:
for i in df.month.unique():
    print(i,(df[(df['month']==i) & (df.obsat==1)].shape[0]),(df[(df['month']==i) & (df.obsat==0)].shape[0]))    

In [ ]:
for i in df.dayofweek.unique():
    print(i,(df[(df['dayofweek']==i) & (df.obsat==1)].shape[0]),(df[(df['dayofweek']==i) & (df.obsat==0)].shape[0]))    

In [ ]:
for i in df.year.unique():
    print (i)
    print(df[(df['year']==i) & (df.obsat==1)].shape[0])
    print(df[(df['year']==i) & (df.obsat==0)].shape[0])
    


# native location column


In [ ]:
df["natloc"].unique()

In [ ]:
df['natloc'].replace({'Delhi /NCR':'Delhi','- Cochin- ':'Cochin'},inplace=True)

In [ ]:
df['natloc'].unique()

# client

In [ ]:
df['client'].replace({'Hospira':'H','Aon Hewitt':'A','UST':'U','Standard Chartered Bank':'S','ANZ':'A1','Pfizer':'P','Standard Chartered Bank Chennai':'S','Aon hewitt Gurgaon':'A','Astrazeneca':'As','Flextronics':'F','Prodapt':'P1','Williams Lea':'W','Barclays':'B','Hewitt':'A','Woori Bank':'Wb'},inplace=True)

# Meeting 

In [ ]:
for i in df["meeting"].unique():
            print(i,(df[(df['meeting']==i) & (df.obsat==1)].shape[0]),(df[(df['meeting']==i) & (df.obsat==0)].shape[0]),sep=" ")    



In [ ]:
df.meeting.isnull().sum()


In [ ]:
df.meeting.replace({'Yes':'Y','Na':'N','No':'N','yes':'Y','Not sure':'N','Not Sure':'N','cant Say':'N'},inplace=True)


In [ ]:
df.drop(df.index[df.meeting.isnull()],axis=0,inplace=True)

In [ ]:
for i in df["meeting"].unique():
            print(i,(df[(df['meeting']==i) & (df.obsat==1)].shape[0]),(df[(df['meeting']==i) & (df.obsat==0)].shape[0]),sep=" ")    

# Industry

In [ ]:
df['industry'].replace({'Pharmaceuticals':'P','IT Services':'I','IT Products and Services':'I','Electronics':'E','Telecom':'E','IT':'I','BFSI':'B'},inplace=True)

# Permit

In [ ]:
for i in df["permit"].unique():
            print(i,(df[(df['permit']==i) & (df.obsat==1)].shape[0]),(df[(df['permit']==i) & (df.obsat==0)].shape[0]),sep=" ")    

In [ ]:
df.drop(df.index[df.permit.isnull()],axis=0,inplace=True)
df.permit.replace({'Yes':'Y','No':'N','Not yet':'N','Yet to confirm':'N','yes':'Y','Na':'N'},inplace=True)

In [ ]:
for i in df["permit"].unique():
            print(i,(df[(df['permit']==i) & (df.obsat==1)].shape[0]),(df[(df['permit']==i) & (df.obsat==0)].shape[0]),sep=" ")    

# Resume

In [ ]:
for i in df["resume"].unique():
            print(i,(df[(df['resume']==i) & (df.obsat==1)].shape[0]),(df[(df['resume']==i) & (df.obsat==0)].shape[0]),sep=" ")    

In [ ]:
df.resume.isnull().sum()


In [ ]:
df['resume'].replace({'Yes':'Y', 'No':'N', 'No- will take it soon':'N', 'Not yet':'N', 'yes':'Y', 'Na':'N',
       'Not Yet':'N'},inplace=True)

In [ ]:
df.drop(df.index[df.resume.isnull()],axis=0,inplace=True)


In [ ]:
for i in df["resume"].unique():
            print(i,(df[(df['resume']==i) & (df.obsat==1)].shape[0]),(df[(df['resume']==i) & (df.obsat==0)].shape[0]),sep=" ")    

# Expat

In [ ]:
for i in df["expat"].unique():
            print(i,(df[(df['expat']==i) & (df.obsat==1)].shape[0]),(df[(df['expat']==i) & (df.obsat==0)].shape[0]),sep=" ")    

In [ ]:
df['expat'].replace({'Yes':'Y','No':'N','yes':'Y','11:00 AM':'Y','10.30 Am':'Y'},inplace=True)


In [ ]:
for i,series in df.iterrows():
        if((series.expat=='Uncertain') & (series.obsat==1)):
            df.loc[i,'expat']='Y'
        elif ((series.expat=='Uncertain') &( series.obsat==0)):
            df.loc[i,'expat']='N'

In [ ]:
for i in df["expat"].unique():
            print(i,(df[(df['expat']==i) & (df.obsat==1)].shape[0]),(df[(df['expat']==i) & (df.obsat==0)].shape[0]),sep=" ")    

# letter

In [ ]:
for i in df["letter"].unique():
            print(i,(df[(df['letter']==i) & (df.obsat==1)].shape[0]),(df[(df['letter']==i) & (df.obsat==0)].shape[0]),sep=" ")    

In [ ]:
df['letter'].replace({'Yes':'Y', 'Havent Checked':'Y', 'No':'N', 'Need To Check':'N', 'Not sure':'Y','Not Sure':'N', 'Not yet':'N', 'no':'N', 'yes':'Y', 'Na':'N'},inplace=True)

In [ ]:
for i in df["letter"].unique():
            print(i,(df[(df['letter']==i) & (df.obsat==1)].shape[0]),(df[(df['letter']==i) & (df.obsat==0)].shape[0]),sep=" ")    

#  Clear

In [ ]:
for i in df["clear"].unique():
            print(i,(df[(df['clear']==i) & (df.obsat==1)].shape[0]),(df[(df['clear']==i) & (df.obsat==0)].shape[0]),sep=" ")    

In [ ]:
df['clear'].replace({'Yes':'Y', 'No':'N', 'No- I need to check':'N', 'yes':'Y', 'Na':'N', 'no':'N'},inplace=True)

In [ ]:
for i in df["clear"].unique():
            print(i,(df[(df['clear']==i) & (df.obsat==1)].shape[0]),(df[(df['clear']==i) & (df.obsat==0)].shape[0]),sep=" ")    

# No.

In [ ]:
for i in df["no."].unique():
            print(i,(df[(df['no.']==i) & (df.obsat==1)].shape[0]),(df[(df['no.']==i) & (df.obsat==0)].shape[0]),sep=" ")    

In [ ]:
df['no.'].replace({'Yes':'Y', 'No':'N', 'No I have only thi number':'N', 'yes':'Y', 'Na':'N'},inplace=True)

In [ ]:
for i in df["no."].unique():
            print(i,(df[(df['no.']==i) & (df.obsat==1)].shape[0]),(df[(df['no.']==i) & (df.obsat==0)].shape[0]),sep=" ")    

 # Call

In [ ]:
for i in df["call"].unique():
            print(i,(df[(df['call']==i) & (df.obsat==1)].shape[0]),(df[(df['call']==i) & (df.obsat==0)].shape[0]),sep=" ")    

In [ ]:
df['call'].replace({'Yes':'Y', 'No':'N', 'No Dont':'Y', 'yes':'Y', 'Na':'N'},inplace=True)

In [ ]:
for i in df["call"].unique():
            print(i,(df[(df['call']==i) & (df.obsat==1)].shape[0]),(df[(df['call']==i) & (df.obsat==0)].shape[0]),sep=" ")    

# Currloc

In [ ]:
df.currloc.replace({'Chennai':'C','Gurgaon':'G','Bangalore':'B','Hyderabad':'H','Delhi':'D','chennai':'C','- Cochin- ':'K','Noida':'N','CHENNAI':'C','chennai ':'C'},inplace=True)

# Venue

In [ ]:
df.venue.replace({'Hosur':'H', 'Gurgaon':'G', 'Bangalore':'B', 'Chennai':'C', 'Hyderabad':'H',
       '- Cochin- ':'K', 'Noida':'N'},inplace=True)

# Jobloc

In [ ]:
df.jobloc.replace({'Hosur':'H', 'Bangalore':'B', 'Chennai':"C", 'Gurgaon':'G', 'Visakapatinam':'V',
       '- Cochin- ':'K', 'Noida':'N'},inplace=True)

# Type

In [ ]:
df['type'].replace({'Scheduled Walkin':'SW', 'Scheduled ':'S', 'Walkin':'W', 'Scheduled Walk In':'SW',
       'Walkin ':'W'},inplace=True)

# Location

In [ ]:
df.location.replace({'Chennai':'C', 'Gurgaon':'G', 'Bangalore':'B', 'Hyderabad':'H', 'Gurgaonr':'G',
       'Delhi':'D', 'chennai':'C', '- Cochin- ':'K', 'Noida':'N', 'CHENNAI':'C', 'chennai ':'C'},inplace=True)

# Closedpos

In [ ]:
df.closedpos.replace({'Production- Sterile':'P','Selenium testing':'S','Dot Net':'D','AML':'A','Trade Finance':'T','Routine':'R','Niche':'N'},inplace=True)

In [ ]:
df.head()

#### Dropping the id column as this is similar to the serial number

In [ ]:
df.drop("id",axis=1,inplace=True)

In [ ]:
df.head()

#### Label encoding all columns except date,skills and obseved attendance

In [ ]:
for i in df.drop(["date","skills","obsat"],axis=1):
    df.loc[:,i]=le.fit_transform(df.loc[:,i])

In [ ]:
df.info()

# Feature importance 

###### Seeing the feature importance using random forest classifier

In [ ]:
X=df.drop(["skills","date","obsat"],axis=1)
y=df["obsat"]

In [ ]:
def get_importance(X,y):
    rf=ensemble.RandomForestClassifier()
    rf.fit(X,y)
    return pd.DataFrame(rf.feature_importances_,columns=["Importance"],index=X.columns).sort_values("Importance",ascending=False)


### Function to plot graph to see how Precision varies with no of features using DECISION TREE (RFECV)

In [ ]:
def modelstats1(X,y):
    np.random.seed(42)
    Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=.25,random_state=42)
    stats=[]
    modelnames=["LR","DecisionTree","KNN","NB"]
    models=list()
    models.append(linear_model.LogisticRegression(C=10))
    models.append(tree.DecisionTreeClassifier())
    models.append(neighbors.KNeighborsClassifier())
    models.append(naive_bayes.GaussianNB())
    for name,model in zip(modelnames,models):
        if name=="KNN":
            k=[l for l in range(5,17,2)]
            grid={"n_neighbors":k}
            grid_obj = GridSearchCV(estimator=model,param_grid=grid,scoring="f1")
            grid_fit =grid_obj.fit(Xtrain,ytrain)
            model = grid_fit.best_estimator_
            model.fit(Xtrain,ytrain)
            name=name+"("+str(grid_fit.best_params_["n_neighbors"])+")"
            print(grid_fit.best_params_)
        else:
            model.fit(Xtrain,ytrain)
        trainprediction=model.predict(Xtrain)
        testpredicti
        (metrics.recall_score(ytrain,trainprediction))
        scores.append(metrics.roc_auc_score(ytrain,trainprediction))
        stats.append(scores)
        scores=list()
        scores.append(name+"-test")
        scores.append(metrics.accuracy_score(ytest,testprediction))
        scores.append(metrics.precision_score(ytest,testprediction))
        scores.append(metrics.recall_score(ytest,testprediction))
        scores.append(metrics.roc_auc_score(ytest,testprediction))
        stats.append(scores)
    
    colnames=["MODELNAME","ACCURACY","PRECISION","RECALL","AUC"]
    return pd.DataFrame(stats,columns=colnames)

### Function to plot graph to see how Precision varies with no. of features using Random Forest Classifier

In [ ]:
imp=get_importance(X,y)
imp

In [ ]:
def rfe_selection(X,y):
    np.random.seed(42)
    rf=ensemble.RandomForestClassifier()
    train=[]
    test=[]
    loss=[]
    m=[rf]
    s=["Random Forest"]
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.25,random_state=42)
    for j in range(0,len(m)):
        #print(s[j])
        model=m[j]
        for i in range(1,30):
            rfecv=feature_selection.RFE(estimator=model,n_features_to_select=i)
            rfecv.fit(X_train,y_train)
            cols=X.columns[rfecv.get_support()]
            X_=X_train[cols]
            X_t=X_test[cols]
            rfecv.fit(X_,y_train)
            
            #model.fit(X_,y)
            #print(cols)
            train.append(metrics.precision_score(y_train,rfecv.predict(X_)))
            test.append(metrics.precision_score(y_test,rfecv.predict(X_t)))
            loss.append(metrics.log_loss(y_test,rfecv.predict_proba(X_t)))
    plt.figure()
    plt.plot(np.arange(1,30,1),train)
    plt.plot(np.arange(1,30,1),test)
    plt.title("Optimal Number of Features (Random Forest) ")
    plt.legend(['train score', 'test score'], loc='upper left')
    plt.ylim(.7,.8)
    plt.show()

### Function to plot graph to see how Precision varies with no of features using DECISION TREE (RFECV)

In [ ]:
from sklearn import feature_selection
def get_feat_dt(X,y):
    np.random.seed(42)
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.25,random_state=42)
    model=tree.DecisionTreeClassifier()
    n=X.shape[1]
    tr_score=[]
    ts_score=[]
    for i in range(1,n):
        rfecv=feature_selection.RFE(model,n_features_to_select=i)
        rfecv.fit(X_train,y_train)
        col=X.columns[rfecv.get_support()]
        model.fit(X_train[col],y_train)
        tr_p=model.predict(X_train[col])
        ts_p=model.predict(X_test[col])
        tr_score.append(metrics.precision_score(y_train,tr_p))
        ts_score.append(metrics.precision_score(y_test,ts_p))
    plt.figure()
    plt.plot(range(1,n),tr_score)
    plt.plot(range(1,n),ts_score)
    plt.xlabel("min_features using Decison Tree")
    plt.ylabel("Precision")
    plt.legend(['train score', 'test score'], loc='upper left')
    plt.ylim(.7,.8)
    plt.show()
    

### Function to calculate the scores of different ML models on the label encoded Data

In [ ]:
def modelstats1(X,y):
    np.random.seed(42)
    Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=.25,random_state=42)
    stats=[]
    modelnames=["LR","DecisionTree","KNN","NB"]
    models=list()
    models.append(linear_model.LogisticRegression(C=10))
    models.append(tree.DecisionTreeClassifier())
    models.append(neighbors.KNeighborsClassifier())
    models.append(naive_bayes.GaussianNB())
    for name,model in zip(modelnames,models):
        if name=="KNN":
            k=[l for l in range(5,17,2)]
            grid={"n_neighbors":k}
            grid_obj = GridSearchCV(estimator=model,param_grid=grid,scoring="f1")
            grid_fit =grid_obj.fit(Xtrain,ytrain)
            model = grid_fit.best_estimator_
            model.fit(Xtrain,ytrain)
            name=name+"("+str(grid_fit.best_params_["n_neighbors"])+")"
            print(grid_fit.best_params_)
        else:
            model.fit(Xtrain,ytrain)
        trainprediction=model.predict(Xtrain)
        testprediction=model.predict(Xtest)
        scores=list()
        scores.append(name+"-train")
        scores.append(metrics.accuracy_score(ytrain,trainprediction))
        scores.append(metrics.precision_score(ytrain,trainprediction))
        scores.append(metrics.recall_score(ytrain,trainprediction))
        scores.append(metrics.roc_auc_score(ytrain,trainprediction))
        stats.append(scores)
        scores=list()
        scores.append(name+"-test")
        scores.append(metrics.accuracy_score(ytest,testprediction))
        scores.append(metrics.precision_score(ytest,testprediction))
        scores.append(metrics.recall_score(ytest,testprediction))
        scores.append(metrics.roc_auc_score(ytest,testprediction))
        stats.append(scores)
    
    colnames=["MODELNAME","ACCURACY","PRECISION","RECALL","AUC"]
    return pd.DataFrame(stats,columns=colnames)

### Function to calculate the score for the RandomForestClassifier models with different number of features

In [ ]:
def run_rf(X,y,i):
    np.random.seed(42)
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.25,random_state=42)
    model=ensemble.RandomForestClassifier()
    rfecv=feature_selection.RFE(model,n_features_to_select=i)
    rfecv.fit(X_train,y_train)
    col=X.columns[rfecv.get_support()]
    model.fit(X_train[col],y_train)
    tr_p=model.predict(X_train[col])
    ts_p=model.predict(X_test[col])
    print (col)
    print(i)
    print("Train Set")
    print("precison",metrics.precision_score(y_train,tr_p)*100)
    print("recall",metrics.recall_score(y_train,tr_p)*100)
    print("accuracy",metrics.accuracy_score(y_train,tr_p)*100)
    print("log loss",metrics.log_loss(y_train,model.predict_proba(X_train[col])))
    print("Test Set")
    print("precision",metrics.precision_score(y_test,ts_p)*100)
    print("recall",metrics.recall_score(y_test,ts_p)*100)
    print("acuuracy",metrics.accuracy_score(y_test,ts_p)*100)
    print("log loss",metrics.log_loss(y_test,model.predict_proba(X_test[col])))
    
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, thresholds = roc_curve(ts_p, y_test)
    roc_auc = auc(fpr, tpr)
    name="RF"
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',  label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    st='Receiver operating characteristic '+name
    plt.title(st)
    plt.legend(loc="lower right")
    plt.show()

###  Function to calculate the score for the RandomForestClassifier models with different number of features

In [ ]:
def run_dt(X,y,i):
    np.random.seed(42)
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.25,random_state=42)
    model=tree.DecisionTreeClassifier()
    rfecv=feature_selection.RFE(model,n_features_to_select=i)
    rfecv.fit(X_train,y_train)
    col=X.columns[rfecv.get_support()]
    model.fit(X_train[col],y_train)
    tr_p=model.predict(X_train[col])
    ts_p=model.predict(X_test[col])
    print(i)
    print("Train Set")
    print("precison",metrics.precision_score(y_train,tr_p)*100)
    print("recall",metrics.recall_score(y_train,tr_p)*100)
    print("accuracy",metrics.accuracy_score(y_train,tr_p)*100)
    print("log loss",metrics.log_loss(y_train,model.predict_proba(X_train[col])))
    print("Test Set")
    print("precision",metrics.precision_score(y_test,ts_p)*100)
    print("recall",metrics.recall_score(y_test,ts_p)*100)
    print("acuuracy",metrics.accuracy_score(y_test,ts_p)*100)
    print("log loss",metrics.log_loss(y_test,model.predict_proba(X_test[col])))
    
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, thresholds = roc_curve(ts_p, y_test)
    roc_auc = auc(fpr, tpr)
    name="DT"
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',  label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    st='Receiver operating characteristic '+name
    plt.title(st)
    plt.legend(loc="lower right")
    plt.show()


### Graph of optimal no. of features in DecisionTreeClassifier

In [ ]:
get_feat_dt(df.drop(["date","obsat","skills"],axis=1),df.obsat)

### Graph of optimal no of features using RFE selection method

In [ ]:
rfe_selection(df.drop(["date","skills","obsat"],axis=1),df.obsat)

### Running modelstats1 on all features of X

In [ ]:
modelstats1(df.drop(["date","obsat","skills"],axis=1),df.obsat)

### Running RandomForestClassifier on 8 features of X

In [ ]:
run_rf(df.drop(["date","obsat","skills"],axis=1),df.obsat,8)

### Running RandomForestClassifier on 9 features of X

In [ ]:
run_rf(df.drop(["date","obsat","skills"],axis=1),df.obsat,9)

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(, ytest)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange',  label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
st='Receiver operating characteristic '+name
plt.title(st)
plt.legend(loc="lower right")
plt.show()
